In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten, Dropout, GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
from tensorflow.keras.optimizers import Adam
import cv2 as cv
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import glob

### Zip Extraction

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/intel-image-classification.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

### Paths

In [ ]:
train_dir = "/content/drive/MyDrive/Datasets/2_intel/images/intel/seg_train"
val_dir = "/content/drive/MyDrive/Datasets/2_intel/images/intel/seg_test"
test_dir = "/content/drive/MyDrive/Datasets/2_intel/images/intel/inference_set_resize"

### preprocessing using image_dataset_from_directory

In [ ]:
label_mode = 'int'
IMG_SIZE = 150
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = train_dir,
    labels='inferred',
    label_mode = label_mode,
    batch_size=64,
    image_size=(IMG_SIZE,IMG_SIZE)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = val_dir,
    labels='inferred',
    label_mode = label_mode,
    batch_size=64,
    image_size=(IMG_SIZE,IMG_SIZE)
)

test_ds = keras.utils.image_dataset_from_directory(
    directory = test_dir,
    labels='inferred',
    label_mode = label_mode,
    batch_size=64,
    image_size=(IMG_SIZE,IMG_SIZE)
)



# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)
test_ds = test_ds.map(process)



Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


## Model Building

#### Variables mostly used

In [ ]:
IMG_SIZE = 150
NO_OF_CLASSES = 6

###  Model Using Transfer VGG19

In [ ]:
conv_base = tf.keras.applications.VGG19(input_shape=(IMG_SIZE,IMG_SIZE,3), classes=NO_OF_CLASSES ,weights="imagenet", pooling=None, include_top=False)

80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
# conv_base.trainable = True

# set_trainable = False

# for layer in conv_base.layers:
#   if layer.name == 'expanded_conv_10':
#     set_trainable = True
#   if set_trainable:
#     layer.trainable = True
#   else:
#     layer.trainable = False

# for layer in conv_base.layers:
#   print(layer.name,layer.trainable)

In [ ]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
# model.add(Dense(2048,activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(6,activation='softmax'))

In [ ]:
# model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
# "categorical_crossentropy"
model.compile(optimizer=Adam(),loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])


from tensorflow.keras.callbacks import ModelCheckpoint
# checkpoint
filepath="/content/drive/MyDrive/Datasets/2_intel/best_trained_models/VGG19_weights-improvement-{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(train_ds,epochs=2,validation_data=validation_ds,callbacks=callbacks_list)


Epoch 1/2
220/220 [==============================] - ETA: 0s - loss: 1.4960 - accuracy: 0.4415
Epoch 1: val_accuracy improved from -inf to 0.51500, saving model to /content/drive/MyDrive/Datasets/2_intel/best_trained_models/VGG19_weights-improvement-01-0.51.h5
220/220 [==============================] - 2166s 9s/step - loss: 1.4960 - accuracy: 0.4415 - val_loss: 1.1663 - val_accuracy: 0.5150
Epoch 2/2
220/220 [==============================] - ETA: 0s - loss: 1.1501 - accuracy: 0.5346
Epoch 2: val_accuracy improved from 0.51500 to 0.53867, saving model to /content/drive/MyDrive/Datasets/2_intel/best_trained_models/VGG19_weights-improvement-02-0.54.h5
220/220 [==============================] - 118s 535ms/step - loss: 1.1501 - accuracy: 0.5346 - val_loss: 1.1110 - val_accuracy: 0.5387


In [ ]:
history = model.fit(train_ds,epochs=40,validation_data=validation_ds,callbacks=callbacks_list)

Epoch 1/40
220/220 [==============================] - ETA: 0s - loss: 1.0363 - accuracy: 0.5747
Epoch 1: val_accuracy did not improve from 0.53867
220/220 [==============================] - 117s 529ms/step - loss: 1.0363 - accuracy: 0.5747 - val_loss: 3926.7673 - val_accuracy: 0.4693
Epoch 2/40
220/220 [==============================] - ETA: 0s - loss: 0.9451 - accuracy: 0.6139
Epoch 2: val_accuracy did not improve from 0.53867
220/220 [==============================] - 118s 532ms/step - loss: 0.9451 - accuracy: 0.6139 - val_loss: 1.2096 - val_accuracy: 0.5360
Epoch 3/40
220/220 [==============================] - ETA: 0s - loss: 0.8328 - accuracy: 0.6722
Epoch 3: val_accuracy did not improve from 0.53867
220/220 [==============================] - 118s 532ms/step - loss: 0.8328 - accuracy: 0.6722 - val_loss: 3.4208 - val_accuracy: 0.2563
Epoch 4/40
220/220 [==============================] - ETA: 0s - loss: 0.7281 - accuracy: 0.7227
Epoch 4: val_accuracy improved from 0.53867 to 0.55900,

### To evaluate on Test data

In [ ]:
test_dir = '/content/drive/MyDrive/Datasets/2_intel/images/intel/inference_set_resize'

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Datasets/2_intel/best_trained_models/final/NNL_weights-improvement-27-0.91.h5')

In [ ]:
test_ds = keras.utils.image_dataset_from_directory(
    directory = test_dir,
    labels='inferred',
    label_mode = 'int',
    batch_size=64,
    image_size=(331,331)
)



# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

# train_ds = train_ds.map(process)
# validation_ds = validation_ds.map(process)
test_ds = test_ds.map(process)

Found 3000 files belonging to 6 classes.


In [ ]:
print("Evaluate on test data")
results = model.evaluate(test_ds,batch_size=64)
print("test loss, test acc:", results)

Evaluate on test data


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


47/47 [==============================] - 609s 10s/step - loss: 0.8248 - sparse_categorical_accuracy: 0.7773
test loss, test acc: [0.8247969746589661, 0.7773333191871643]


In [ ]:
base = tf.keras.applications.EfficientNetV2L(
    include_top=False,
    weights="imagenet",
    include_preprocessing=True
)

473176280/473176280 [==============================] - 7s 0us/step


In [ ]:
base.summary()

Model: "efficientnetv2-l"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling_2 (Rescaling)        (None, None, None,   0           ['input_3[0][0]']                
                                3)                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, None, None,   864         ['rescaling_2[0][0]']            
                                32)                                                

In [ ]:
test_dir = '/content/drive/MyDrive/Datasets/2_intel/best_trained_models/final/NNL_weights-improvement-27-331-64-0.91.h5'

In [ ]:
model = keras.models.load_model(test_dir)